#### 中文数据集
https://modelscope.cn/datasets/testUser/GSM8K_zh

#### 模型与显存
1. Qwen2.5-0.5B
2. 实测占用22G显存


##### 小红书：AI有温度，日更学习资料&面试资料
##### 作者：Tiger

视频讲解地址：http://xhslink.com/n/6nxt7jc9IUN

In [ ]:
import re
import torch
import pandas as pd
from datasets import load_dataset, Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer
from modelscope.msdatasets import MsDataset

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [ ]:
# 使用中文GSM8K数学数据集
data =  MsDataset.load('testUser/GSM8K_zh'
                 , subset_name='default'
                 , split='train'
                 , cache_dir='/data/zhuantai/R1')
data = pd.DataFrame(data)

In [ ]:
data = data[['question_zh', 'answer_only']]

In [ ]:
def convert_data(data):
    new_data = []
    for x in data.values.tolist():
        # 构建训练数据格式
        item = {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': x[0]}
            ],
            'answer':x[1]
        }
        
        new_data.append(item)
    return new_data

dataset = convert_data(data)

In [ ]:
# 从文本中提取 <answer> 标签内的内容
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

# 奖励函数：检查回答的正确性
def correctness_reward_func(prompts, completions, answer, **kwargs):
    # completions：模型生成的候选答案集合
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    # 如果提取的回答与标准答案一致，奖励 2.0，否则奖励 0.0
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

# 奖励函数：检查回答是否为整数
def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    # 如果提取的回答是一个数字字符串，奖励 0.5，否则奖励 0.0
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

# 奖励函数：严格检查完成结果的格式
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    # 定义严格的格式匹配模式，要求 <reasoning> 和 <answer> 标签内有换行
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    # 如果匹配成功，奖励 0.5，否则奖励 0.0
    return [0.5 if match else 0.0 for match in matches]

# 奖励函数：宽松检查完成结果的格式
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    # 定义宽松的格式匹配模式，不要求 <reasoning> 和 <answer> 标签内有换行
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    # 如果匹配成功，奖励 0.5，否则奖励 0.0
    return [0.5 if match else 0.0 for match in matches]

# 计算文本中特定 XML 标签的数量并给予相应奖励
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

# 奖励函数：根据 XML 标签计数给予奖励
def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    # 从 completions 中提取每个完成结果的内容
    contents = [completion[0]["content"] for completion in completions]
    # 对每个内容调用 count_xml 函数计算奖励
    return [count_xml(c) for c in contents]

In [ ]:
# 模型地址
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
# 输出地址
output_dir="outputs/Qwen-0.5B-GRPO"
# 自定义任务名称
run_name="Qwen-0.5B-GRPO-gsm8k-zh"

training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=1,
    # 梯度累积步数，将多个小批次的梯度累积起来，等效于使用更大的批次大小进行训练
    gradient_accumulation_steps=4,
    # 每个样本生成的候选数量，例如在生成式任务中可能会生成多个候选输出
    num_generations=16,
    max_prompt_length=256,
    max_completion_length=200,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    log_on_each_node=False,
    use_vllm=False,
    vllm_gpu_memory_utilization=.3,
    vllm_device="cuda:0",
    report_to="none"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=None
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func],
    args=training_args,
    train_dataset=dataset,
    #peft_config=peft_config
)
trainer.train()

trainer.save_model(output_dir)